# Get Candlestick Patterns

## Import relevant libraries

In [7]:
import os
import pandas as pd
import talib
import mplfinance as mpf
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Get the data

In [8]:
input_data_path = '../data'
input_data_filename = 'binance_BTCUSDT_1m_from_2020_01_01_to_2021_12_31'
input_data_extension = ".csv"
full_path_input_data = os.path.join(input_data_path, input_data_filename + input_data_extension)

In [9]:
df = pd.read_csv(full_path_input_data)

In [10]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volumne', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volumne',
       'ignore', 'formatted_open_time', 'formatted_close_time', 'upper_shadow',
       'lower_shadow', 'real_body'],
      dtype='object')

## Identify Candlestick Patterns

In [11]:
candle_names = talib.get_function_groups()['Pattern Recognition']

- Inverted Hammer --> Single [Bullish]
- Hammer --> Single [Bullish]
- Bullish Engulfing (not available at ta-lib) --> Double [Bullish]
- Piercing Pattern --> Double [Bullish]
- Morning Star --> Triple [Bullish]


- Shooting Star --> Single [Bearish]
- Hanging Man --> Single [Bearish]
- Bearish Engulfing (not available at ta-lib) --> Double [Bearish]
- Dark Cloud Cover --> Double [Bearish]
- Evening Star --> Triple [Bearish]

- Engulfing [positive: Bullish / negative: Bearish]

In [12]:
candle_names = [
    'CDLINVERTEDHAMMER',
    'CDLHAMMER',
    'CDLPIERCING',
    'CDLMORNINGSTAR',
    'CDLSHOOTINGSTAR',
    'CDLHANGINGMAN',
    'CDLDARKCLOUDCOVER',
    'CDLEVENINGSTAR',
    'CDLENGULFING'
]

In [13]:
# extract OHLC 
op = df['open']
hi = df['high']
lo = df['low']
cl = df['close']

# create columns for each pattern
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = getattr(talib, candle)(op, hi, lo, cl)

In [14]:
df.sample(10)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,real_body,CDLINVERTEDHAMMER,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING
484835,1606984620000,19397.77,19421.01,19397.13,19407.11,91.840250,1606984679999,1.782287e+06,1551,49.407312,...,9.34,0,0,0,0,0,0,0,0,0
645410,1616646660000,52341.97,52410.64,52339.69,52390.96,27.318531,1616646719999,1.430873e+06,834,12.419877,...,48.99,0,0,0,0,0,0,0,0,0
370427,1600116540000,10685.35,10693.84,10685.35,10692.56,23.383975,1600116599999,2.499926e+05,232,16.665999,...,7.21,0,0,0,0,0,0,0,0,0
275012,1594391640000,9180.01,9186.23,9178.69,9186.23,29.732606,1594391699999,2.730143e+05,462,20.876058,...,6.22,0,0,0,0,0,0,0,0,0
438755,1604216220000,13730.77,13730.77,13724.84,13729.27,7.961068,1604216279999,1.092830e+05,200,3.149576,...,1.50,0,0,0,0,0,0,0,0,0
501675,1607995020000,19342.53,19346.99,19336.06,19336.55,76.437818,1607995079999,1.478457e+06,773,14.822238,...,5.98,0,0,0,0,0,0,0,0,0
16336,1578816960000,8120.16,8124.00,8117.02,8121.29,28.085677,1578817019999,2.280829e+05,430,17.984128,...,1.13,0,0,0,0,0,0,0,0,0
452625,1605048420000,15359.34,15362.45,15357.62,15359.96,22.446335,1605048479999,3.447812e+05,727,11.918016,...,0.62,0,0,0,0,0,0,0,0,0
969155,1636120800000,61773.25,61850.00,61773.25,61847.67,21.910580,1636120859999,1.354513e+06,1179,13.840480,...,74.42,0,0,0,0,0,0,0,0,0
883689,1630985640000,52757.24,52778.83,52738.15,52750.07,46.065800,1630985699999,2.429968e+06,1229,16.926070,...,7.17,0,0,0,0,0,0,0,0,0


## Split 'CDLENGULFING' into Bullish and Bearish one

In [17]:
list_open_time_bullish_engulfing = df[df['CDLENGULFING']==100]['open_time'].to_list()
list_open_time_bearish_engulfing = df[df['CDLENGULFING']==-100]['open_time'].to_list()

In [18]:
df['CDLENGULFINGBULLISH'] = np.where(df['open_time'].isin(list_open_time_bullish_engulfing), 100, 0)
df['CDLENGULFINGBEARISH'] = np.where(df['open_time'].isin(list_open_time_bearish_engulfing), -100, 0)

### Check

In [19]:
df[df['CDLENGULFING']==100].sample(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING,CDLENGULFINGBULLISH,CDLENGULFINGBEARISH
334147,1597939740000,11825.43,11832.31,11822.04,11831.43,59.811417,1597939799999,7.072909e+05,761,50.537791,...,0,0,0,0,0,0,0,100,100,0
185253,1588993500000,9868.62,9871.87,9863.39,9870.18,23.238107,1588993559999,2.293313e+05,392,12.040643,...,0,0,0,0,0,0,0,100,100,0
391585,1601386020000,10740.54,10744.79,10740.54,10742.34,17.009906,1601386079999,1.827227e+05,248,9.858196,...,0,0,0,0,0,0,0,100,100,0
233633,1591896300000,9322.46,9338.99,9316.54,9335.19,179.628809,1591896359999,1.675983e+06,1788,101.469980,...,0,0,0,0,0,0,0,100,100,0
717963,1621025880000,49230.01,49359.07,49224.60,49300.01,74.356935,1621025939999,3.665496e+06,1914,43.759704,...,0,0,0,0,0,0,0,100,100,0


In [20]:
df[df['CDLENGULFING']==-100].sample(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING,CDLENGULFINGBULLISH,CDLENGULFINGBEARISH
519898,1609105800000,26417.03,26449.19,26385.60,26392.29,73.933415,1609105859999,1.952643e+06,1473,29.800493,...,0,0,0,0,0,0,0,-100,0,-100
981031,1636833360000,63975.55,63975.55,63931.01,63931.02,8.375080,1636833419999,5.355677e+05,483,3.645970,...,0,0,0,0,0,0,0,-100,0,-100
101695,1583971020000,7933.00,7934.64,7925.00,7925.00,25.134545,1583971079999,1.993193e+05,251,9.411963,...,0,0,0,0,0,0,0,-100,0,-100
1017601,1639027560000,49726.45,49728.49,49700.23,49710.59,5.032500,1639027619999,2.501939e+05,320,2.077120,...,0,0,0,0,0,0,0,-100,0,-100
722478,1621296780000,44173.42,44185.21,44065.23,44091.18,80.869540,1621296839999,3.567367e+06,1904,37.690320,...,0,0,0,0,0,0,0,-100,0,-100


## Export the Data

### Export parameters

In [204]:
export_path = "../data"
export_filename = input_data_filename + \
                  "_candlesticks_signals_raw"
export_extension = ".csv"
full_export_path = os.path.join(export_path, export_filename + export_extension)

### Export

In [205]:
df.to_csv(full_export_path, index=False)